### Human-In-The-Loop

There are 3 main situations where human in the loop might be needed are:

- When there is an involvement of sensitive actions that need approval.
- When there is missing context
- When we need to debug agents

In [1]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [2]:
# Load the model
from langchain_ollama import ChatOllama
# Load the model
model = ChatOllama(model="qwen3:1.7b", 
                   temperature=0,
                   validate_model_on_init=True,
                   seed=42)

In [4]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [6]:
from pprint import pprint

pprint(response)

print(response['__interrupt__'])

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Response '
                                                                          'to '
                                                                          'your '
                                                                          'email.'},
                                                         'description': 'Tool '
                                                                        'execution '
                                                                        'requires '
                                                                        'approval\n'
                                                                        '\n'
                                                                        'Tool: '
                                                                        'send_email\n'
                                                                        'Args: '
       

In [7]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Response to your email.


We can approve, Reject or Edit the agent response. 

- Approve

In [8]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='f526fe22-5ae1-41e2-97e5-960d3102672d'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:1.7b', 'created_at': '2025-12-27T16:51:52.7532115Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14280218800, 'load_duration': 3767862900, 'prompt_eval_count': 188, 'prompt_eval_duration': 584305100, 'eval_count': 694, 'eval_duration': 9548880000, 'logprobs': None, 'model_name': 'qwen3:1.7b', 'model_provider': 'ollama'}, id='lc_run--019b60b9-0b61-7b50-871d-367627d65469-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'd7ced88a-f0dd-454c-9ccb-6b2d3bfc9edf', 'type': 'tool_call'}, {'name': 'send_email', 'args': {'body': 'Response to your email.'}, 'id': 'a506d3c1-d322-46b0-90af-c1d22156384d', 'type': 't

- Reject

In [9]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='f526fe22-5ae1-41e2-97e5-960d3102672d'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:1.7b', 'created_at': '2025-12-27T16:51:52.7532115Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14280218800, 'load_duration': 3767862900, 'prompt_eval_count': 188, 'prompt_eval_duration': 584305100, 'eval_count': 694, 'eval_duration': 9548880000, 'logprobs': None, 'model_name': 'qwen3:1.7b', 'model_provider': 'ollama'}, id='lc_run--019b60b9-0b61-7b50-871d-367627d65469-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'd7ced88a-f0dd-454c-9ccb-6b2d3bfc9edf', 'type': 'tool_call'}, {'name': 'send_email', 'args': {'body': 'Response to your email.'}, 'id': 'a506d3c1-d322-46b0-90af-c1d22156384d', 'type': 't

- Edit

In [14]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='f526fe22-5ae1-41e2-97e5-960d3102672d'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:1.7b', 'created_at': '2025-12-27T16:51:52.7532115Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14280218800, 'load_duration': 3767862900, 'prompt_eval_count': 188, 'prompt_eval_duration': 584305100, 'eval_count': 694, 'eval_duration': 9548880000, 'logprobs': None, 'model_name': 'qwen3:1.7b', 'model_provider': 'ollama'}, id='lc_run--019b60b9-0b61-7b50-871d-367627d65469-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'd7ced88a-f0dd-454c-9ccb-6b2d3bfc9edf', 'type': 'tool_call'}, {'name': 'send_email', 'args': {'body': 'Response to your email.'}, 'id': 'a506d3c1-d322-46b0-90af-c1d22156384d', 'type': 't